In [1]:
import os
import pandas as pd
import numpy as np
import math
import datetime as dt

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

In [4]:
import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [22]:
##Bitcoin historical data from Yahoo Finance
bth=pd.read_csv('BTC-USD.csv')

In [23]:
print('Total number of fields present in the dataset: ',bth.shape[1])

Total number of fields present in the dataset:  7


In [24]:
bth.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-11-05,7404.520020,7617.479980,7333.189941,7407.410156,7407.410156,2380410112
1,2017-11-06,7403.220215,7445.770020,7007.310059,7022.759766,7022.759766,3111899904
2,2017-11-07,7023.100098,7253.319824,7023.100098,7144.379883,7144.379883,2326340096
3,2017-11-08,7141.379883,7776.419922,7114.020020,7459.689941,7459.689941,4602200064
4,2017-11-09,7446.830078,7446.830078,7101.520020,7143.580078,7143.580078,3226249984


In [25]:
bth.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
1822,2022-11-01,20494.898438,20647.289063,20359.845703,20485.273438,20485.273438,39819303159
1823,2022-11-02,20482.958984,20742.810547,20087.134766,20159.503906,20159.503906,55552169483
1824,2022-11-03,20162.689453,20382.095703,20086.240234,20209.988281,20209.988281,43228750179
1825,2022-11-04,20208.769531,21209.560547,20188.019531,21147.230469,21147.230469,64072727950
1826,2022-11-05,21165.160156,21445.326172,21106.419922,21385.876953,21385.876953,63250665472


In [26]:
bth.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1827.000000,1827.000000,1827.000000,1827.000000,1827.000000,1.827000e+03
mean,20174.549205,20688.375810,19597.262743,20179.757670,20179.757670,2.605953e+10
std,17057.423307,17497.276645,16544.297489,17050.258045,17050.258045,1.975678e+10
min,3236.274658,3275.377930,3191.303467,3236.761719,3236.761719,2.326340e+09
25%,7570.877685,7761.965088,7440.585205,7578.637940,7578.637940,1.058802e+10
50%,10696.691406,10947.041992,10393.099609,10725.599609,10725.599609,2.378866e+10
75%,33767.375000,34778.076172,32265.797851,33734.515625,33734.515625,3.544966e+10
max,67549.734375,68789.625000,66382.062500,67566.828125,67566.828125,3.509679e+11


In [27]:
print('Null Values:',maindf.isnull().values.sum())
print('NA values:',maindf.isnull().values.any())
# maindf=maindf.dropna()

Null Values: 0
NA values: False


In [28]:
##EDA

In [32]:
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')

y_overall = maindf.loc[(maindf['Date'] >= '2017-11-05')
                     & (maindf['Date'] <= '2022-11-05')]

y_overall.drop(y_overall[['Adj Close','Volume']],axis=1)

,Date,Open,High,Low,Close
0,2021-11-05,61460.078125,62541.468750,60844.609375,61125.675781
1,2021-11-06,61068.875000,61590.683594,60163.781250,61527.480469
2,2021-11-07,61554.921875,63326.988281,61432.488281,63326.988281
3,2021-11-08,63344.066406,67673.742188,63344.066406,67566.828125
4,2021-11-09,67549.734375,68530.335938,66382.062500,66971.828125
...,...,...,...,...,...
361,2022-11-01,20494.898438,20647.289063,20359.845703,20485.273438
362,2022-11-02,20482.958984,20742.810547,20087.134766,20159.503906
363,2022-11-03,20162.689453,20382.095703,20086.240234,20209.988281
364,2022-11-04,20208.769531,21209.560547,20188.019531,21147.230469


In [33]:
monthvise= y_overall.groupby(y_overall['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

,Open,Close
Date,,
January,41368.073463,41114.422379
February,40591.103934,40763.474051
March,41889.148438,41966.237525
April,41694.653646,41435.319662
May,31900.711127,31706.105217
June,24783.338477,24383.685482
July,21424.733052,21539.253843
August,22471.866557,22366.266318
September,19821.353711,19804.779232


In [35]:
names = cycle(['Bitcoin Open Price','Bitcoin Close Price','Stock High Price','Stock Low Price'])

fig = px.line(y_overall, x=y_overall.Date, y=[y_overall['Open'], y_overall['Close'], 
                                          y_overall['High'], y_overall['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

In [47]:
##LSTM to predict Closing Price
closebth = bth[['Date','Close']]

In [48]:
closebth = closebth[closebth['Date'] > '2021-11-05']
close_stock = closebth.copy()
print("Total data for prediction: ",closebth.shape[0])

Total data for prediction:  365


In [49]:
closebth.head(10)

,Date,Close
1462,2021-11-06,61527.480469
1463,2021-11-07,63326.988281
1464,2021-11-08,67566.828125
1465,2021-11-09,66971.828125
1466,2021-11-10,64995.230469
1467,2021-11-11,64949.960938
1468,2021-11-12,64155.941406
1469,2021-11-13,64469.527344
1470,2021-11-14,65466.839844
1471,2021-11-15,63557.871094


In [50]:
##Scaling data
del closebth['Date']
scaler=MinMaxScaler(feature_range=(0,1))
closebth=scaler.fit_transform(np.array(closebth).reshape(-1,1))
print(closebth.shape)

(365, 1)


In [53]:
training_size=int(len(closebth)*0.60)
test_size=len(closebth)-training_size
train_data,test_data=closebth[0:training_size,:],closebth[training_size:len(closebth),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (219, 1)
test_data:  (146, 1)


In [54]:
##Time Series Forecasting

In [56]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]    
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [57]:
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

In [59]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (203, 15, 1)
X_test:  (130, 15, 1)


In [60]:
##Model Build

In [61]:
model=Sequential()
model.add(LSTM(10,input_shape=(None,1),activation="relu"))
model.add(Dense(1))
model.compile(loss="mean_squared_error",optimizer="adam")

In [63]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=32,verbose=1)

Epoch 1/200
7/7 [==============================] - 0s 14ms/step - loss: 0.0023 - val_loss: 0.0112
Epoch 2/200
7/7 [==============================] - 0s 11ms/step - loss: 0.0023 - val_loss: 0.0112
Epoch 3/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0023 - val_loss: 0.0111
Epoch 4/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0023 - val_loss: 0.0109
Epoch 5/200
7/7 [==============================] - 0s 10ms/step - loss: 0.0023 - val_loss: 0.0110
Epoch 6/200
7/7 [==============================] - 0s 10ms/step - loss: 0.0023 - val_loss: 0.0110
Epoch 7/200
7/7 [==============================] - 0s 10ms/step - loss: 0.0023 - val_loss: 0.0105
Epoch 8/200
7/7 [==============================] - 0s 11ms/step - loss: 0.0023 - val_loss: 0.0109
Epoch 9/200
7/7 [==============================] - 0s 11ms/step - loss: 0.0023 - val_loss: 0.0109
Epoch 10/200
7/7 [==============================] - 0s 11ms/step - loss: 0.0022 - val_loss: 0.0105
Epoch 11/200
7/7 [===

7/7 [==============================] - 0s 9ms/step - loss: 0.0019 - val_loss: 0.0069
Epoch 85/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0019 - val_loss: 0.0067
Epoch 86/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0019 - val_loss: 0.0067
Epoch 87/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0019 - val_loss: 0.0070
Epoch 88/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0019 - val_loss: 0.0065
Epoch 89/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0019 - val_loss: 0.0068
Epoch 90/200
7/7 [==============================] - 0s 16ms/step - loss: 0.0019 - val_loss: 0.0065
Epoch 91/200
7/7 [==============================] - 0s 19ms/step - loss: 0.0018 - val_loss: 0.0067
Epoch 92/200
7/7 [==============================] - 0s 15ms/step - loss: 0.0018 - val_loss: 0.0066
Epoch 93/200
7/7 [==============================] - 0s 10ms/step - loss: 0.0018 - val_loss: 0.0066
Epoch 94/200
7/7 [===========

7/7 [==============================] - 0s 10ms/step - loss: 0.0015 - val_loss: 0.0035
Epoch 167/200
7/7 [==============================] - 0s 13ms/step - loss: 0.0014 - val_loss: 0.0032
Epoch 168/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0014 - val_loss: 0.0033
Epoch 169/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0014 - val_loss: 0.0029
Epoch 170/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0014 - val_loss: 0.0030
Epoch 171/200
7/7 [==============================] - 0s 10ms/step - loss: 0.0014 - val_loss: 0.0027
Epoch 172/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0014 - val_loss: 0.0030
Epoch 173/200
7/7 [==============================] - 0s 10ms/step - loss: 0.0015 - val_loss: 0.0027
Epoch 174/200
7/7 [==============================] - 0s 10ms/step - loss: 0.0014 - val_loss: 0.0027
Epoch 175/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0014 - val_loss: 0.0024
Epoch 176/200
7/7 [

In [64]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

5/5 [==============================] - 0s 2ms/step


((203, 1), (130, 1))

In [65]:
##Eval

In [66]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [68]:
##Variance Regression
print("Train data explained variance regression score:", 
      explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", 
      explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9436274735064291
Test data explained variance regression score: 0.6877448838856965


In [71]:
##R square score
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9427214192949845
Test data R2 score: -0.7926534162272869


In [73]:
##Regression loss,MGD,MPD
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.001767208525699606
Test data MGD:  0.00999043236563078
----------------------------------------------------------------------
Train data MPD:  70.43251247957348
Test data MPD:  215.48044835154244


In [77]:
look_back=time_step
trainPredictPlot = np.empty_like(closebth)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closebth)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closebth)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['Date'],
                       'original_close': close_stock['Close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (365, 1)
Test predicted data:  (365, 1)


In [80]:
##Predicting next 30 days
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 30
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        
        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        
        lst_output.extend(yhat.tolist())
        i=i+1

In [83]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closebth[len(closebth)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 30 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 30 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()